# たし算
量子コンピュータには従来のコンピュータと同じ計算もできるという特徴があります。たし算の回路を確認します。

## 今回学ぶこと
1. 量子ゲートを使って二進数のたし算を実装
2. 量子の重ね合わせを使って1つの回路で複数のたし算

## Blueqatのインストール
pipからBlueqatをインストールします。

In [2]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## 二進数のたし算
たし算は桁上がりを実行するccxゲートと、位の足し合わせを実行するcxゲートを使います。今回はa+b=cdという二進数のたし算の量子回路を行います。今回はaとbの値によって4種類のたし算を実装します。それぞれのたし算は、

0+0 = 00 => 0000  
0+1 = 01 => 0101  
1+0 = 01 => 1001  
1+1 = 10 => 1110  

となります。4量子ビットのビット列で表現し、前半の2量子ビットが入力値aとbで、後半の2量子ビットが出力値cとdです。aとbを入力する回路と、実際にたし合わせを実現する回路を別に作り、何度か使い回します。1のようなデータの入力はXゲートを使って0を反転させて実装します。

たし算の回路部分はこのようになります。*はコントロールビットです。

```
a ---*---*------- a
b ---*---|---*--- b
0 ---X---|---|--- c
0 -------X---X--- d
```

aとbにXゲートを使ってデータを入れると、ccx回路で桁上がり、cx回路で位のたし合わせを行います。

In [6]:
#ツールの読み込み
from blueqat import Circuit

#たし算部分を実現します。
adder = Circuit().ccx[0,1,2].cx[0,3].cx[1,3]

In [13]:
#0+0
(Circuit() + adder).m[:].run(shots=100)

Counter({'0000': 100})

In [14]:
#0+1
(Circuit().x[1] + adder).m[:].run(shots=100)

Counter({'0101': 100})

In [15]:
#1+0
(Circuit().x[0] + adder).m[:].run(shots=100)

Counter({'1001': 100})

In [16]:
#1+1
(Circuit().x[0,1] + adder).m[:].run(shots=100)

Counter({'1110': 100})

このように、たし算が実装できました。

## 重ね合わせを利用したたし算
ここで、Xゲートでデータを一つ一つ入れる代わりに、Hゲートを使ってたし算をしてみます。

In [17]:
#Xゲートの代わりにHゲートを入力に使う
(Circuit().h[0,1] + adder).m[:].run(shots=100)

Counter({'0101': 27, '1110': 30, '0000': 24, '1001': 19})

アダマールゲートを利用すると4つのたし算が大体1/4ずつ答えとして出てきました。このように汎用たし算回路を作ると、重ね合わせ状態を利用した計算を実行できます。

## もつれを利用したたし算
次にHゲートの代わりにa+b=1となるたし算を量子のもつれを使って行ってみます。

In [19]:
#01と10のもつれを作る
(Circuit().h[0].cx[0,1].x[0] + adder).m[:].run(shots=100)

Counter({'0101': 41, '1001': 59})

このように、入力値が01と10がもつれているので、この2つのたし算が約1/2ずつ出てきます。

--------

## 解説：回路の作成（１桁目）
まずは上の1桁目から作成します。
1桁目を見ると、0, 0, 0, 1 の順になっています。

表で表すと以下になります。

|X|Y|X+Yの2桁目|
|:-:|:-:|:-:|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|

これを見ると CCXゲートと同じことがわかります。
(CCXは入力の頭2つのビットが1ならば3番目のビットを反転させる)

このことから入力が 11 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_0.png" width="23%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（２桁目）
次にもう一つのくらいを作成します。
2桁目を見ると、0, 1, 1, 0 の順になっています。

表で表すと以下になります。

|X|Y|X+Yの1桁目|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

これを見ると CXゲートと同じことがわかります。
(左辺がCXの入力で右辺が出力の2番目のビット)

このことから入力が 10 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_1.png" width="27%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（全体）
最後に上の二つの回路をまとめます。

入力を 00 としてまとめた回路を以下に示します。

<img src="./img/008/008_02_2.png" width="30%">

上2ビットを入力、残りの2ビットが出力です。
始めの CCX は1桁目の部分で残りの 2つの CX は2桁目の部分です。

## 解説：重ね合わせ回路の実装
量子の重ね合わせを用いると4パターンを一気に操作することができます。

式としては以下の物を考えます。

<img src="./img/009/009_02_0.png" width="35%">

見てわかる通り右辺に入力の4パターンが出てきました。
この性質を用いて実装します。

回路は以下のようになります。

<img src="./img/009/009_02_1.png" width="35%">

今まで Xゲートを施していた部分を Hゲートにして 4パターンを作っています。

## (応用) 一般の足し算
一般の10進数同士で足し算を実装します。  
$a, b$ の和を考えることとします。
$a, b$ は $a = a_n ... a_0$, $b = b_n ... b_0$ と2進数で表示できます。  
(ここで n は数が大きい方の位を基準とします。)

回路は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

$c_i$ を carry ビットと言って繰り上がった数を表します。  
足し算回路は carry と呼ばれる繰り上がりの部分と sum の合計の部分にわかれています。

まずはこの2つの部分を考察します。

### 繰り上がり
回路は以下のようになります。

<img src="./img/100_0.png" width="30%">

回路を上から $c_i, a_i, b_i, c_{i+1}$ とすると $c_{i+1}$ に繰り上がりの部分が現れます。

### 合計
各位の和を考えます。回路は以下のようになります。

<img src="./img/100_1.png" width="30%">

回路を上から $c_i, a_i, b_i$ とすると $b_i$ に3つの数の和の繰り上がりを除いた部分が現れます。

### 実装
足し算回路を実装します。

流れは以下のようになります。  
1. carry 回路で各位の繰り上がりを求める。  
2. CXゲートで最後のcarryの部分を元に戻す。  
3. sum 回路で $b_n$ に各位の和を出力する。  
4. carry の逆回路でその位の値を元に戻す。  
5. sum 回路で $b_i$ に各位の和を出力する。  
6. 4,5の繰り返し。

以上の流れから $a+b$ は $b_{n+1} ... b_0$ の部分に出力されることが分かります。

足し算回路を実装するために下準備をします。  
まずは carry とその逆回路, sum 回路を作ります。

In [1]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum(i):
    return Circuit().cx[i+1,i+2].cx[i,i+2]

10進数を2進数にする関数も作っておきます。

In [4]:
def tobinary(A):
    return bin(A)[2:]

tobinary(10)

'1010'

数を回路にマッピングする関数を作ります。

In [5]:
def digits(a,b): 
     # 2進数に変換
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # nを決めて大きい方にビット数を合わせる
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # マッピング
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

digits(2,2)

'0000110'

回路の初期状態は全て0なので、マッピングした値に合うようにXゲートを施す必要があります。  
この関数を作ります。

In [7]:
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

toX("101").m[:].run(shots=100)

Counter({'101': 100})

最後に出力の部分を考えます。  
まずは出力は2進数なので10進数に変換する関数を考えます。

In [9]:
def todecimal(A):
    return int(str(A),2) 

todecimal(1001)

9

回路では $a_i, b_i, c_i$ が混ざった値が出力されるのでその中で $b_i$ のみ取り出します。

In [12]:
def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

getb("0000110")

2

### 一般化回路
以上で全ての準備が整ったので足し算回路を作ります。

In [15]:
def plus(a,b): 
     # 2進数表示のマッピング
     qubits = len(digits(a,b)) 
     cir1 = toX(digits(a,b)) 
     digi = int((len(digits(a,b))-1)/3) 

     # 前半のcarry回路
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     # 最後の桁の処理
     cir3 = Circuit(qubits).cx[-3,-2] + sum((digi-1)*3) 

     # carryの逆回路とsum回路でbiに和を出
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     result = (cir1 + cir2 + cir3 + cir4).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

実際に計算してみます。

In [19]:
plus(2,2)

4

In [20]:
plus(13,15)

28

In [21]:
plus(70,90)

160

最後の計算は時間かかりますが、一般の加算器を実装できました。

### 参考文献
https://arxiv.org/pdf/quant-ph/9511018.pdf